# Creating Cutouts of Roman Data with Astrocut

---

# Kernel Information

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

# Imports

The following packages are used for these reasons:

- *asdf* - To handle ASDF input and output
- *roman_datamodels* - To read Roman WFI ASDF files
- *s3fs* - To access cloud files as though they were local
- *matplotlib* - The plotting package used for demonstrating the cutout
- *numpy* - To handle array functions
- *astrocut* - The main package import, used to generate the cutouts from the input file
- *astropy* - To manage the output FITS data file type and handle input sky coordinates

In [ ]:
import asdf
import s3fs
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy

from astrocut import get_center_pixel, asdf_cut
import astropy.units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.utils.exceptions import AstropyDeprecationWarning
import roman_datamodels as rdm

import warnings

# Ignore warnings from the 'astropy.wcs' module
# When creating cutouts -> WARNING: Polynomial distortion is not implemented.
warnings.filterwarnings('ignore', module='astropy.wcs')

# Ignore AstropyDeprecationWarning
# When creating cutouts -> WARNING: AstropyDeprecationWarning: The class "Fits" has been renamed to "FITS" in version 7.0. 
# The old name is deprecated and may be removed in a future version. Use FITS instead. [astropy.units.format]
warnings.filterwarnings('ignore', category=AstropyDeprecationWarning)

## Introduction

The `astrocut` tool allows users to generate cutouts out of large astronomical images. In this tutorial, we demonstrate how to use `astrocut` to generate cutouts from a calibrated Level 2 (L2; calibrated rate image) simulated product created with Roman I-Sim.

---

## Setup

The first step of the analysis is to read the Roman WFI image data, which are stored in Advanced Scientific Data Format (ASDF) files. For this example, we start with a calibrated L2 simulated image created with [Roman I-Sim](https://romanisim.readthedocs.io/en/latest/). For more information about Roman's data products, check the [Roman User Documentation](https://roman-docs.stsci.edu/).

After launch, Roman data will be available through the [Mikulski Archive for Space Telescopes (MAST)](https://archive.stsci.edu/). For testing purposes, some simulated data has been placed in a separate S3 bucket.  We access the Roman data files from the RRN S3 bucket using `s3fs`. For more information about how to access the S3 bucket, please see the [Data Discovery and Access](../data_discovery_and_access/data_discovery_and_access.ipynb) tutorial.

In [ ]:
fs = s3fs.S3FileSystem()
asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/AAS_WORKSHOP/'
asdf_file_uri = asdf_dir_uri + 'r0003201001001001004_0001_wfi01_f106_cal.asdf'

# Read the file into memory
with fs.open(asdf_file_uri, 'rb') as f:
    with asdf.open(f) as af:
        dm = rdm.open(af)
        data = deepcopy(dm.data)
        gwcs = deepcopy(dm.meta.wcs)

## Examine Coordinates

### GWCS

Roman ASDF images use a [Generalized World Coordinate System (GWCS)](https://gwcs.readthedocs.io/en/latest/index.html) object to handle coordinate transformations, as opposed to the [Flexible Image Transport System (FITS)](https://fits.gsfc.nasa.gov/fits_primer.html) [WCS standard](https://fits.gsfc.nasa.gov/fits_wcs.html). GWCS is more flexible than FITS WCS, and contains a compound model of transformations from detector to world coordinates.  It is stored in the image files within the `meta.wcs` attribute.  Let's examine the sky and pixel coordinates of our image:

In [ ]:
print('some gwcs info:')
print('---------------')
print('- input frame: ', gwcs.input_frame)
print('- output frame: ', gwcs.output_frame)
print('- pixel bounds: ', gwcs.bounding_box)
print('- sky footprint: ', gwcs.footprint())

### Coordinate Analysis

Next, we will do some coordinate analysis of points in the image. For three sets of input sky coordinates, we will compute the pixel coordinates, the computed sky coordinates, and the separation between the input and computed sky coordinates. The points correspond to the center of the image, the far upper right, and the far lower left. First, let's define some helper functions:

In [ ]:
# Define some helper functions

def get_sep(coord1, coord2):
    """ Compute separation between two sky coordinates """
    return coord1.separation(coord2).to(u.arcsec)
    

def get_pix_info(xx, coord):
    """ Print some pixel coordinate info """
    print('Pixel Coord:', xx)
    ss = gwcs(*xx, with_units=True)
    print('Computed Sky Coord:', ss)
    print('Separation:', get_sep(coord, ss))    


def print_coord_info(coord):
    """ Print some sky coordinate info """
    print('Input Sky Coord:', coord)
    xx = gwcs.invert(coord)
    get_pix_info(xx, coord)

Next, we will do some coordinate analysis for the center of the image, the far upper-right corner, and the far lower-left corner in the image. For three sets of input sky coordinates, we will compute the pixel coordinates, the computed sky coordinates, and the separation between the input and computed sky coordinates.

In [ ]:
# sky coord of central pixel
print('Central Image Pixel')
print('-------------------')
pix_x, pix_y = np.shape(data)[0] // 2, np.shape(data)[1] // 2
center_coord = gwcs(pix_x, pix_y, with_units=True)
print_coord_info(center_coord)
print('\n')

# sky coord at far edge (upper right)
print('Far Edge Pixel - Upper Right')
print('-----------------------------')
ss = gwcs(4077, 4077, with_units=True)
print_coord_info(ss)
print('\n')

# sky coord at far edge (lower left)
print('Far Edge Pixel - Lower Left')
print('---------------------------')
ss = gwcs(2, 2, with_units=True)
print_coord_info(ss)
print('\n')

# get computed pixel coordinates from world coordinates with get_center_pixel
test_coord = SkyCoord('270.8719 -0.16437', unit='deg') # test out different sky coordinates here!
print('Computed Closet Pixel Coordinate')
print('--------------------------------')
pc, ww = get_center_pixel(gwcs, test_coord.ra, test_coord.dec)
get_pix_info(pc, test_coord)

---

## Create Image Cutout in FITS Format
Here, we will create the image cutout as a FITS file.  First, we plot the expected cutout on the input image with a red square.

In [ ]:
# define cutout size
cutout_size = 200

# define cutout coordinates
coord = center_coord

# plot expected cutout on original image
pc, ww = get_center_pixel(gwcs, coord.ra, coord.dec)
plt.imshow(data, vmin=0, vmax=5, origin='lower')
tmp = Cutout2D(data, position=pc, wcs=ww, size=cutout_size, mode='partial')
tmp.plot_on_original(color='red')
plt.title('Expected Cutout Position')
padding = 500
plt.xlim(pix_x - padding, pix_x + padding)
plt.ylim(pix_y - padding, pix_y + padding)
plt.show()

The ``asdf_cut`` function takes as input the following: 
- `input_file`: The input ASDF filename or URI
- `ra`: The right ascension of the central cutout
- `dec`: The declination of the central cutout
- `cutout_size`: The image cutout pixel size (optional, default `200`)
- `output_file`: The name of the output cutout file. The output format is determined by the filename extension (optional, default `"example_roman_cutout.fits"`)
- `write_file`: Flag to write the cutout to a file or not (optional, default `True`)
- `fill_value`: The fill value for any pixels outside the original image (optional, default `np.nan`)
  
Let's create a square cutout, at the center of the image, 200 pixels on a side, and save the output to a new FITS file.

In [ ]:
# create the image cutout as a FITS file
cutout = asdf_cut(asdf_file_uri, coord.ra, coord.dec, cutout_size=cutout_size, output_file="roman-demo.fits")

### Inspect FITS Cutout Image
Now let's inspect the cutout a bit and display the image cutout with matplotlib.  The ``asdf_cut`` function returns an [Astropy Cutout2D](https://docs.astropy.org/en/stable/nddata/utils.html#overview) object.  

In [ ]:
print('shape:', cutout.shape)
print('orginal position:', cutout.position_original)
print('cutout position:', cutout.position_cutout)

In [ ]:
with fits.open("roman-demo.fits") as hdulist:
    hdulist.info()
    header = hdulist[0].header
    img = hdulist[0].data

#### Cutout in Pixel Coordinates

In [ ]:
def plot_cutout_pixel(img):
    """ Display the cutout using pixel coordinates """
    plt.figure(figsize=(6, 6))
    plt.imshow(img, vmin=0, vmax=5, origin="lower")
    plt.title('Cutout in Pixel Coordinates')
    plt.show()


plot_cutout_pixel(img)

#### Cutout in Sky Coordinates

In [ ]:
def plot_cutout_world(cutout, img, coord, wcs):    
    """ Display the cutout using world coordinate system """
    plt.figure(figsize=(6, 6))
    plt.title('Cutout in Sky Coordinates')
    plt.xticks([])
    plt.yticks([])
    ax = plt.subplot(projection=wcs)
    ax.imshow(img, vmin=0, vmax=5, origin='lower')
    ax.grid(color='white', ls='solid')
    
    # overplot the original sky coordinate
    ax.scatter_coord(coord, s=100, edgecolor='cyan', facecolor='none')
    
    # plot central pixel
    center_loc = img.shape[0] // 2
    ax.scatter([center_loc], [center_loc], s=100, edgecolor='white', facecolor='none')
    
    # plot sky coord of central pixel
    cc = SkyCoord(*cutout.wcs.all_pix2world([center_loc], [center_loc], 0), unit=u.degree)
    ax.scatter_coord(cc, s=50, edgecolor='yellow', facecolor='none')
    
    plt.show()

In [ ]:
# print the cutout wcs
wcs = WCS(header)
print(wcs, '\n')

# print FITS WCS position of various sky coordinates
print("SkyCoords Info:")
print('Input Requested', coord)

# plot the figure with wcs
plot_cutout_world(cutout, img, coord, wcs)

---

## Create Image Cutout in ASDF Format
Now, we will create the image cutout as an ASDF file. We will create the cutout with the same parameters as the FITS version to illustrate that both cutouts contain the same data.

In [ ]:
# define cutout size
cutout_size = 200

# define cutout coordinates
coord = center_coord

# create the image cutout as an ASDF file
cutout_asdf = asdf_cut(asdf_file_uri, coord.ra, coord.dec, cutout_size=cutout_size, output_file="roman-demo.asdf")

### Inspect ASDF Cutout Image

In [ ]:
print('shape:', cutout_asdf.shape)
print('orginal position:', cutout_asdf.input_position_original)
print('cutout position:', cutout_asdf.input_position_cutout)

#### Cutout in Pixel Coordinates

In [ ]:
plot_cutout_pixel(cutout_asdf.data)

#### Cutout in Sky Coordinates

In [ ]:
# print the cutout wcs
print(cutout_asdf.wcs, '\n')

# print FITS WCS position of various sky coordinates
print("SkyCoords Info:")
print('Input Requested', coord)

# plot the figure with wcs
plot_cutout_world(cutout_asdf, cutout_asdf.data, coord, cutout_asdf.wcs)

---

## Partial Image Cutouts

Sometimes, cutouts are made near the edge of Roman images. In these cases, part of the cutout may fall outside of the original image.

The `asdf_cut` function takes an optional `fill_value` parameter for this circumstance. It describes the value that should be given to any pixels in the cutout that fall outside of the input image. The default value is `np.nan`.

To illustrate this, we will take a cutout from the lower right corner of the image.

In [ ]:
# Roman images are 4088 x 4088, so pick values for edge_x and edge_y that are greater than 4088 - cutout_size
cutout_size = 300
edge_x = 4000
edge_y = 4000

# convert to sky coordinates
edge_coord = gwcs(edge_x, edge_y, with_units=True)
edge_coord

# plot expected cutout on original image
pc, ww = get_center_pixel(gwcs, edge_coord.ra, edge_coord.dec)
plt.imshow(data, vmin=0, vmax=5, origin='lower')
tmp = Cutout2D(data, position=pc, wcs=ww, size=cutout_size, mode='partial')
tmp.plot_on_original(color='red')
plt.title('Expected Cutout Position')
padding = 500
plt.xlim(edge_x - padding, edge_x + padding)
plt.ylim(edge_y - padding, edge_y + padding)
plt.show()

Notice how the top and right side of the square are outside of the image? We can assign the value of these pixels in our cutout using the `fill_value` parameter.

### Create Image Cutout

We will create the partial cutout as a FITS file, and we will fill in any outside pixels with a value of 0.

In [ ]:
# create the image cutout as a FITS file
cutout_partial = asdf_cut(asdf_file_uri, edge_coord.ra, edge_coord.dec, cutout_size=cutout_size, 
                          fill_value=0, output_file="partial-cut.fits")

### Inspect Image Cutout

In [ ]:
print('shape:', cutout_partial.shape)
print('orginal position:', cutout_partial.position_original)
print('cutout position:', cutout_partial.position_cutout, '\n')

with fits.open("partial-cut.fits") as hdulist:
    hdulist.info()
    header_partial = hdulist[0].header
    img_partial = hdulist[0].data

In [ ]:
# plot cutout in pixel coordinates
plot_cutout_pixel(img_partial)

In [ ]:
# print the cutout wcs
wcs = WCS(header_partial)
print(wcs, '\n')

# print FITS WCS position of various sky coordinates
print("SkyCoords Info:")
print('Input Requested', edge_coord)

# plot the figure with wcs
plot_cutout_world(cutout_partial, img_partial, edge_coord, wcs)

As expected, the pixels that fall outside of the original image have been assigned a value of 0.

---

## Additional Resources
- [Astrocut Documentation](https://astrocut.readthedocs.io)
- [Advanced Scientific Data Format Documentation](https://asdf.readthedocs.io/en/latest/)

---

## About this Notebook

**Authors**: Thomas Dutkiewicz, Brian Cherinka, Sam Bianco<br>
**Last Updated**: 2025-01-10

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 